In [1]:
import torch

In [ ]:
def upSampling(origins, angles, ground_truth_distance, num_roll):

    # check if angle between pairs are small 
    diff = torch.abs(angles - torch.roll(angles, 1,0))
    temp = diff < 0.003 #0.003radient approxximately 0.05 degree
    mask_small_ang = temp.all(dim=1)

    # for rays with small angle between, we can sample points close to origin, but with direction on other ray
    
    # check if origin between pairs are small
    diff2 = torch.abs(origins - torch.roll(origins,1,0))
    temp = diff2 < 0.5
    mask_small_org = temp.all(dim=1)
    
    # now find if the target location is close
    dir = sph2cart(angles)
    pos = dir * ground_truth_distance
    diff3 = torch.abs(pos - torch.roll(pos, 1, 0))
    temp = diff3 < 0.5
    mask_small_dis = temp.all(dim=1)
    
    # create a list of magnitudes with even spacing from 0 to 1
    t = torch.linspace(0,1, 10).expand(dir_x.shape[0], 10)  # [batch_size, num_bins]
    
    # preterb the spacing
    mid = (t[:, :-1] + t[:, 1:]) / 2.
    lower = torch.cat((t[:, :1], mid), -1)
    upper = torch.cat((mid, t[:, -1:]), -1)
    u = torch.rand(t.shape, device = device)
    t = lower + (upper - lower) * u  # [batch_size, nb_bins]
    t = rearrange(t, 'a b -> b a')  # [num_bins, batch_size]  take transpose so that multiplication can broadcast





    

    return sample_position, sample_directions, sample_gt_distance